<a href="https://colab.research.google.com/github/noor-osama/headPoseEstimation-/blob/main/tan_tan_taa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install mediapipe

In [ ]:
!wget http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
!unzip AFLW2000-3D.zip

--2022-06-10 21:39:06--  http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
Resolving www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)... 159.226.21.60
Connecting to www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)|159.226.21.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87413900 (83M) [application/x-zip-compressed]
Saving to: ‘AFLW2000-3D.zip’

AFLW2000-3D.zip     100%[===================>]  83.36M   325KB/s    in 9m 24s  

2022-06-10 21:48:34 (151 KB/s) - ‘AFLW2000-3D.zip’ saved [87413900/87413900]

Archive:  AFLW2000-3D.zip
   creating: AFLW2000/
   creating: AFLW2000/Code/
  inflating: AFLW2000/Code/DrawSolidHead.m  
  inflating: AFLW2000/Code/DrawTextureHead.m  
  inflating: AFLW2000/Code/main_show_without_BFM.m  
  inflating: AFLW2000/Code/main_show_with_BFM.m  
   creating: AFLW2000/Code/Mex/
  inflating: AFLW2000/Code/Mex/compile.m  
  inflating: AFLW2000/Code/Mex/Tnorm_Vnorm.h  
  inflating: AFLW2000/Code/Mex/Tnorm_VnormC.cpp  
  inflatin

In [ ]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error ,r2_score


In [ ]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow

In [ ]:
mylist = [Path(f).stem for f in glob.glob("AFLW2000/*.mat")]
sio.loadmat(os.path.join('AFLW2000/',mylist[0]))

{'Color_Para': array([[0.70669746, 1.0348098 , 1.1388271 , 0.343065  , 0.08621594,
         0.00307747, 1.        ]], dtype=float32),
 'Exp_Para': array([[-1.0897612 ],
        [ 0.5944782 ],
        [-0.489466  ],
        [ 0.63535964],
        [-0.0580295 ],
        [ 0.33420901],
        [-0.27618136],
        [ 0.99251936],
        [ 0.24649337],
        [ 0.2516629 ],
        [-0.51603132],
        [ 0.08345538],
        [-0.26454622],
        [ 0.02451065],
        [-0.25529621],
        [-0.15018897],
        [-0.23094465],
        [ 0.1060707 ],
        [ 0.15962804],
        [ 0.18041692],
        [ 0.23135913],
        [-0.15746039],
        [-0.13094076],
        [-0.25158844],
        [ 0.21679091],
        [-0.08129298],
        [ 0.08988512],
        [ 0.08544559],
        [-0.22081508]]),
 'Illum_Para': array([[ 1.04675114e+00,  1.12633324e+00,  1.14832306e+00,
          1.54048979e-01,  1.06746880e-02,  8.32199156e-02,
         -2.80801833e-01,  1.17520332e+00,  0.00000

In [ ]:
def get_list_from_filenames(file_path):
    # input:    relative path to .txt file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines

In [ ]:
images = get_list_from_filenames("AFLW2000/*.jpg")
images

['image00595',
 'image00972',
 'image00093',
 'image00428',
 'image02076',
 'image03705',
 'image03804',
 'image02856',
 'image01829',
 'image02449',
 'image03651',
 'image04234',
 'image03984',
 'image01643',
 'image02369',
 'image02846',
 'image02202',
 'image02211',
 'image00070',
 'image03816',
 'image02373',
 'image02011',
 'image04207',
 'image01323',
 'image00825',
 'image01767',
 'image03726',
 'image03186',
 'image02355',
 'image01474',
 'image00642',
 'image01621',
 'image02378',
 'image02272',
 'image04316',
 'image01217',
 'image00841',
 'image00681',
 'image00679',
 'image00923',
 'image03776',
 'image03897',
 'image00501',
 'image00299',
 'image00109',
 'image01879',
 'image03806',
 'image00504',
 'image02696',
 'image00656',
 'image03766',
 'image02834',
 'image01781',
 'image04244',
 'image03322',
 'image01416',
 'image00059',
 'image03835',
 'image01328',
 'image01089',
 'image00872',
 'image00673',
 'image01677',
 'image01765',
 'image03718',
 'image02720',
 'image027

In [ ]:
def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params

In [ ]:
def get_imageData (r):
  random_file = r
  x = []
  y = []
  result = pd.DataFrame()
  xs = ['x'+str(i) for i in range(1,469)]
  ys = ['y'+str(i) for i in range(1,469)]
  labels = ['pitch', 'yaw', 'roll']
  column_names = xs + ys + labels


    
  faceModule = mediapipe.solutions.face_mesh
      # loading image and its correspinding mat file
  with faceModule.FaceMesh(static_image_mode=True,max_num_faces=1) as faces:
          # loading the image
          image = cv2.imread('/content/AFLW2000/'+random_file+'.jpg')
          # processing the face to extract the landmark points (468 point) for each x,y,z
          results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
          if results.multi_face_landmarks != None: 
            # looping over the faces in the image
            for face in results.multi_face_landmarks:
                for landmark in face.landmark:
                    x = pd.DataFrame(np.append(x,landmark.x).reshape(1,-1))
                    y = pd.DataFrame(np.append(y,landmark.y).reshape(1,-1))
                    xy = pd.DataFrame(pd.concat([x, y], join='inner', axis=1))
                    pyr=pd.DataFrame(get_ypr_from_mat('AFLW2000/' + random_file + '.mat').reshape(1,-1))
                    result = pd.DataFrame(pd.concat([xy, pyr], join='inner', axis=1))
                    
  if len(x) == 1 :
    result.columns = column_names  
    #print (result.shape)
    return len(x),result
  else : 
    return 0 ,0
    

In [ ]:
images = get_list_from_filenames("AFLW2000/*.jpg")
final = pd.DataFrame ()
delete_list = []

for img in images :
    x,d = get_imageData (img)
    if x == 1 :
      final = final.append(d, ignore_index=True)
    else :
      idx = images.index(img)
  
final.to_csv('data.csv',index=False)


In [ ]:
data = pd.read_csv('data.csv')
X = data.iloc[:,:-3]
pitch =pd.DataFrame(data.iloc[:,-3])
yaw =pd.DataFrame(data.iloc[:,-2])
roll =pd.DataFrame(data.iloc[:,-1])
X.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,0.510301,0.539955,0.521339,0.526380,0.545696,0.547080,0.544260,0.371067,0.548495,0.554468,...,0.642067,0.636280,0.645506,0.644711,0.646732,0.489658,0.496514,0.502666,0.484771,0.481386
1,0.496543,0.490750,0.494637,0.473451,0.489455,0.489239,0.490431,0.349120,0.488918,0.487339,...,0.612627,0.605239,0.620930,0.614501,0.619865,0.489245,0.494036,0.496846,0.470059,0.463056
2,0.508726,0.506480,0.506519,0.489819,0.505869,0.504833,0.502442,0.387133,0.500955,0.500391,...,0.614937,0.608465,0.615240,0.617361,0.619867,0.496441,0.502198,0.506480,0.486023,0.478864
3,0.430302,0.404915,0.437721,0.429249,0.404812,0.417521,0.459454,0.466700,0.472960,0.472966,...,0.584243,0.580398,0.598994,0.584009,0.590151,0.508752,0.511522,0.512846,0.510750,0.505476
4,0.476873,0.500210,0.492089,0.512359,0.508868,0.520090,0.547259,0.418335,0.568778,0.580176,...,0.637855,0.635989,0.648092,0.639156,0.640425,0.521228,0.522037,0.524026,0.546657,0.543878


In [ ]:
df = X.copy()
df.head()
x_norm = X.loc[:,'x5']
x_norm
y_norm = X.loc[:,'y5']
y_norm

0       0.616022
1       0.584179
2       0.595384
3       0.558316
4       0.607941
          ...   
1848    0.622313
1849    0.602331
1850    0.616200
1851    0.599784
1852    0.547888
Name: y5, Length: 1853, dtype: float64

In [ ]:
import re
for i in df.columns:
  if re.search('x', i):
    df[i] = df[i] - x_norm    
  if re.search('y', i):
    df[i] = df[i] - y_norm

df.head()


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,-0.035394,-0.005740,-0.024356,-0.019315,0.0,0.001385,-0.001435,-0.174629,0.002800,0.008773,...,0.026045,0.020258,0.029485,0.028690,0.030711,-0.126363,-0.119508,-0.113356,-0.131251,-0.134636
1,0.007088,0.001295,0.005182,-0.016004,0.0,-0.000216,0.000976,-0.140335,-0.000537,-0.002117,...,0.028448,0.021060,0.036751,0.030322,0.035686,-0.094934,-0.090143,-0.087333,-0.114120,-0.121123
2,0.002857,0.000611,0.000649,-0.016050,0.0,-0.001037,-0.003427,-0.118736,-0.004914,-0.005478,...,0.019553,0.013080,0.019856,0.021977,0.024482,-0.098944,-0.093186,-0.088904,-0.109361,-0.116521
3,0.025491,0.000104,0.032910,0.024437,0.0,0.012709,0.054643,0.061888,0.068148,0.068155,...,0.025927,0.022083,0.040678,0.025693,0.031836,-0.049564,-0.046794,-0.045470,-0.047566,-0.052840
4,-0.031995,-0.008658,-0.016779,0.003491,0.0,0.011222,0.038390,-0.090533,0.059910,0.071308,...,0.029913,0.028047,0.040150,0.031214,0.032483,-0.086713,-0.085905,-0.083916,-0.061285,-0.064064


In [ ]:
ratio = X.loc[:,'x158'] - X.loc[:,'x150']
ratio
normalized_df = df.copy()
for i in normalized_df.columns:
    normalized_df[i] = normalized_df[i] - ratio  

normalized_df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y459,y460,y461,y462,y463,y464,y465,y466,y467,y468
0,-0.138861,-0.109207,-0.127823,-0.122782,-0.103467,-0.102082,-0.104902,-0.278095,-0.100667,-0.094694,...,-0.077421,-0.083209,-0.073982,-0.074777,-0.072756,-0.229830,-0.222974,-0.216823,-0.234717,-0.238102
1,0.006047,0.000255,0.004142,-0.017044,-0.001040,-0.001256,-0.000064,-0.141376,-0.001577,-0.003157,...,0.027408,0.020019,0.035710,0.029282,0.034645,-0.095974,-0.091183,-0.088373,-0.115160,-0.122163
2,-0.014666,-0.016912,-0.016873,-0.033573,-0.017523,-0.018559,-0.020950,-0.136259,-0.022437,-0.023001,...,0.002030,-0.004442,0.002333,0.004454,0.006959,-0.116466,-0.110709,-0.106427,-0.126884,-0.134043
3,0.013394,-0.011993,0.020813,0.012340,-0.012097,0.000613,0.042546,0.049791,0.056051,0.056058,...,0.013830,0.009986,0.028581,0.013596,0.019739,-0.061661,-0.058891,-0.057567,-0.059663,-0.064937
4,-0.185519,-0.162182,-0.170302,-0.150033,-0.153524,-0.142301,-0.115133,-0.244056,-0.093614,-0.082216,...,-0.123610,-0.125476,-0.113373,-0.122310,-0.121041,-0.240237,-0.239428,-0.237439,-0.214809,-0.217587


In [ ]:

#train_test_spilt
Xp_train, Xp_test, p_train, p_test = train_test_split(normalized_df, pitch, test_size=0.33, random_state=42)
# pitch prediction
p_reg =SVR(C=1.0, epsilon=0.2)
p_reg.fit(Xp_train, p_train)
p_pred = pd.DataFrame(p_reg.predict (Xp_test))
p_pred.columns = ["pitch"]
p_pred
#MSE 
mseP = mean_squared_error(p_test,p_pred)
print("MSE of pitch : %.2f" %mseP)
r2P = r2_score(p_test,p_pred)
print("R2 score of pitch : %.2f" %r2P)

##############################################################
#train_test_spilt
Xy_train, Xy_test, y_train, y_test = train_test_split(normalized_df, yaw, test_size=0.33, random_state=42)
# yaw prediction
y_reg =SVR(C=1.0, epsilon=0.2)
y_reg.fit(Xy_train, y_train)
y_pred = pd.DataFrame(y_reg.predict (Xy_test))
y_pred.columns = ["yaw"]
y_pred
#MSE 
mseY =mean_squared_error(y_test,y_pred)
print("MSE of yaw  : %.2f" %mseY)
r2Y =r2_score(y_test,y_pred)
print("R2 score of yaw : %.2f" % r2Y)

##############################################################
#train_test_spilt
Xr_train, Xr_test, r_train, r_test = train_test_split(normalized_df, roll, test_size=0.33, random_state=42)
# yaw prediction
r_reg =SVR(C=1.0, epsilon=0.2)
r_reg.fit(Xr_train, r_train)
r_pred = pd.DataFrame(r_reg.predict (Xr_test))
r_pred.columns = ["roll"]
r_pred
#MSE 
mseR =mean_squared_error(r_test,r_pred)
print("MSE of rool  : %.2f" %mseR)
r2R =r2_score(r_test,r_pred)
print("R2 score of roll : %.2f" % r2R)



MSE of pitch : 0.03
R2 score of pitch : 0.65
MSE of yaw  : 0.03
R2 score of yaw : 0.92
MSE of rool  : 0.02
R2 score of roll : 0.82
